# Landscape or Cityscape?

The idea behind this notebook is to test out various CNNs to determine which networks are best suited to determining if an picture is an image of a city or an image of nature. My data sources for this experiment are the subreddits r/earthporn and r/cityporn. I chose these subreddits as they are focused and fairly strictly moderated. Each contains only aestetically pleasing pictures of landscapes and cityscapes respectively. I took all the directly linked images from these subreddits since 1/1/17 that have over a certain threshold of upvotes, which gave me a few thousand images that are almost entirely either a picture of a natural landscape or a picture of a cityscape

I will use keras' implementation of VGG16 as feature extration for a few kinds of classifiers.

In [51]:
#general imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [52]:
#keras specific imports
from keras import Model, Input
from keras.layers import Dense, Flatten,GlobalMaxPool2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

from keras.applications import VGG16,vgg16
from keras.applications import ResNet50, resnet50
from keras.applications import InceptionV3, inception_v3

In [53]:
from project_utils import get_test, create_confusion_matrix

In [54]:
from importlib import reload
import project_utils
reload(project_utils)

<module 'project_utils' from '/home/ubuntu/Landscape_or_Cityscape/project_utils.py'>

In [55]:
#loading pretrained model weights
VGG = VGG16(weights = 'imagenet', include_top = False, input_shape= (224,224,3))

RN50 = ResNet50(weights = 'imagenet', include_top = False, input_shape= (224,224,3))

IV3 = InceptionV3(weights = 'imagenet', include_top = False, input_shape= (224,224,3))

In [58]:
# This function takes in headless model (some pretrained CNN)
# and adds a max pooling and two dense fully connected layers to it.
def create_model(base_layers):
    for layer in base_layers.layers:
        layer.trainable = False
    X = base_layers.output
    X = GlobalMaxPool2D()(X)
    X = Dense(200, activation = 'relu', name = '1')(X)
    X = Dense(100, activation = 'relu',name = '2')(X)
    predictions = Dense(2 , activation = 'softmax',name = '3')(X)
    model = Model(inputs = base_layers.inputs, outputs = predictions)
    return model
    

In [59]:
#Preparing data generators
#this keras utilitiy will continously generate images that have been slightly rotated 
#or shifted or flipped from the directories where I stored the images for both training and validation
data_gen = ImageDataGenerator(shear_range= 0.2,
                                    zoom_range= 0.2,
                                    horizontal_flip= True,
                                    width_shift_range=0.2,
                                    height_shift_range = 0.2,
                                    preprocessing_function= (lambda x: x/127.5 - 1))                                           

train_generator = data_gen.flow_from_directory('./images/train', target_size= (224,224), class_mode= 'categorical')
validation_generator = data_gen.flow_from_directory('./images/validation', target_size= (224,224), class_mode= 'categorical')

vgg_model = create_model(VGG)

vgg_model.compile(optimizer='rmsprop',loss = 'categorical_crossentropy',metrics = ['accuracy'])

Found 4850 images belonging to 2 classes.
Found 605 images belonging to 2 classes.


In [60]:
#this will cause the model to stop fitting if the validation loss has gone down for 3 epochs
#this is to prevent overfitting by training the model for too long
early_stopping = EarlyStopping(min_delta = .01, 
                               patience = 3,
                               verbose = 1,
                               mode = 'min')

In [ ]:
vgg_model.fit_generator(generator = train_generator, 
                        epochs=20,
                        steps_per_epoch= 20, #there are 32 images per step so this is 64 per epoch
                        callbacks = [early_stopping],
                        validation_data = validation_generator,
                        validation_steps=10) #validation on 320 images

Epoch 1/20
18/20 [==========================>...] - ETA: 21s - loss: 0.6063 - acc: 0.8003

/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (93153600 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


19/20 [===========================>..] - ETA: 10s - loss: 0.5882 - acc: 0.8059

/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (98130452 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


20/20 [==============================] - 320s 16s/step - loss: 0.5924 - acc: 0.8047 - val_loss: 1.0087 - val_acc: 0.4437
Epoch 2/20
20/20 [==============================] - 278s 14s/step - loss: 0.3160 - acc: 0.8750 - val_loss: 0.2677 - val_acc: 0.9117
Epoch 3/20
20/20 [==============================] - 296s 15s/step - loss: 0.2581 - acc: 0.9219 - val_loss: 0.1946 - val_acc: 0.9500
Epoch 4/20
20/20 [==============================] - 273s 14s/step - loss: 0.2768 - acc: 0.9077 - val_loss: 0.3237 - val_acc: 0.8896
Epoch 5/20
20/20 [==============================] - 296s 15s/step - loss: 0.3042 - acc: 0.8984 - val_loss: 0.3158 - val_acc: 0.8969
Epoch 6/20
20/20 [==============================] - 281s 14s/step - loss: 0.2378 - acc: 0.9094 - val_loss: 0.1783 - val_acc: 0.9369
Epoch 7/20
19/20 [===========================>..] - ETA: 9s - loss: 0.2032 - acc: 0.9424 

In [ ]:
#makes and evalutes the model on the test set
X_test, y_test = project_utils.get_test('_3_10','vgg16')

vgg_model.evaluate(X_test,y_test)

In [ ]:
#makes a confusion matrix for the test set
preds = vgg_model.predict(X_test)

conf_matrix = create_confusion_matrix(y_test,preds,{0:'city',1:'earth'})

conf_matrix

In [ ]:
#function to help with error analysis and sanity checking
def show_and_predict(path,model,modelname):
    pylab.imshow(pylab.imread(path))
    prepped = project_utils.prep_for_model(path,modelname)
    pred = model.predict(prepped)
    print("p(city): {}, p(nature): {}".format(pred[0,0],pred[0,1]))
    

In [ ]:
show_and_predict('images/test/cityporn_3_10/image229.jpg',vgg_model,'vgg16')

In [ ]:
show_and_predict('images/test/earthporn_3_10/image3077.jpg',vgg_model,'vgg16')

In [ ]:
show_and_predict('images/test/cityporn_3_10/image971.jpg',vgg_model,'vgg16')

This network is doing okay, but potentially we can do even better! Now to try out some other base networks 

In [ ]:
iv3_model = create_model(IV3)

iv3_model.compile(optimizer='rmsprop',loss = 'categorical_crossentropy',metrics = ['accuracy'])
vgg_model.fit_generator(generator = train_generator, 
                        epochs=20,
                        steps_per_epoch= 20, #there are 32 images per step so this is 64 per epoch
                        callbacks = [early_stopping],
                        validation_data = validation_generator,
                        validation_steps=10) #validation on 320 images

In [ ]:
X_test, y_test = project_utils.get_test('_3_10','inception_v3')

iv3_model.evaluate(X_test,y_test)

In [ ]:
#makes a confusion matrix for the test set
preds = iv3_model.predict(X_test)

conf_matrix = create_confusion_matrix(y_test,preds,{0:'city',1:'earth'})

conf_matrix

In [ ]:
show_and_predict('images/test/cityporn_3_10/image229.jpg',iv3_model,'inception_v3')

In [ ]:
show_and_predict('images/test/earthporn_3_10/image3077.jpg',iv3_model,'inception_v3')

In [ ]:
show_and_predict('images/test/cityporn_3_10/image971.jpg',iv3_model,'inception_v3')